# Data Cleaning

In [2]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime as dt
import calendar

# increasing max column display in Pandas
pd.set_option('display.max_columns', 54)

In [3]:
# reading in the supply chain csv-
df = pd.read_csv('../data/DataCoSupplyChainDataset.csv', encoding= 'unicode_escape', parse_dates = ['order date (DateOrders)', 'shipping date (DateOrders)'])

In [4]:
# checking the shape of the data
df.shape

(180519, 53)

In [5]:
# checking for missing values-
df.isnull().sum()

Type                                  0
Days for shipping (real)              0
Days for shipment (scheduled)         0
Benefit per order                     0
Sales per customer                    0
Delivery Status                       0
Late_delivery_risk                    0
Category Id                           0
Category Name                         0
Customer City                         0
Customer Country                      0
Customer Email                        0
Customer Fname                        0
Customer Id                           0
Customer Lname                        8
Customer Password                     0
Customer Segment                      0
Customer State                        0
Customer Street                       0
Customer Zipcode                      3
Department Id                         0
Department Name                       0
Latitude                              0
Longitude                             0
Market                                0


In [6]:
# product description and Zip code have too many null values to be useful. These cannot be imputed and will be dropped. 
# Remaining null values are not impactful- 
df.drop(columns = ['Order Zipcode', 'Product Description'], inplace = True)
pass

In [7]:
# dropping the following customer fields as they are not value add and in some cases cause privacy concerns-
df.drop(columns = ['Customer Email', 'Customer Fname', 'Customer Lname', 'Customer Password', 'Customer Street', 'Customer Country'], inplace = True)
pass

In [8]:
# dropping unecessary department features-
df.drop(columns = ['Department Id'], inplace = True)

In [9]:
# dropping unnecessary order features-
df.drop(columns = ['Order Customer Id', 'Order Item Discount', 'Order Item Cardprod Id', 'Order Item Id', 'Sales',
                   'Sales per customer', 'Order Item Total', 'Order Profit Per Order', 'Order Item Product Price', 'Benefit per order'], inplace = True)

In [10]:
# dropping unnecessary product features-
df.drop(columns = ['Product Category Id', 'Product Image', 'Product Status'], inplace = True)

In [11]:
# renaming columns to be more sensible-
df.rename(columns = {'order date (DateOrders)': 'order date', 'shipping date (DateOrders)': 'shipping date',
                     'Days for shipping (real)': 'actual ship days', 'Days for shipment (scheduled)': 'estimated ship days'}, inplace = True)

In [12]:
# updating column names to be lower and snake case-

df.columns= df.columns.str.lower()
df.columns= df.columns.str.replace(' ', '_')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180519 entries, 0 to 180518
Data columns (total 31 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   type                      180519 non-null  object        
 1   actual_ship_days          180519 non-null  int64         
 2   estimated_ship_days       180519 non-null  int64         
 3   delivery_status           180519 non-null  object        
 4   late_delivery_risk        180519 non-null  int64         
 5   category_id               180519 non-null  int64         
 6   category_name             180519 non-null  object        
 7   customer_city             180519 non-null  object        
 8   customer_id               180519 non-null  int64         
 9   customer_segment          180519 non-null  object        
 10  customer_state            180519 non-null  object        
 11  customer_zipcode          180516 non-null  float64       
 12  de

In [14]:
# converting order and ship date columns to a datetime dtype-
df['order_date']= pd.to_datetime(df['order_date'])
df['shipping_date']= pd.to_datetime(df['shipping_date'])

In [15]:
# creating a ship performance metric which factors actual shipping days vs estimated-
df['ship_performance'] = df['actual_ship_days'] - df['estimated_ship_days']

In [16]:
# creating an ontime column. 0 = shipment was late, 1 = shipment arrived on or before estimated delivery date-
df['ontime'] = np.where(df['ship_performance'] < 1, 1 , 0)

In [17]:
# creating an is_fraud column based on whether the 'order status' is 'SUSPECTED_FRAUD'-
df['is_fraud'] = np.where(df['order_status'] == 'SUSPECTED_FRAUD', 1, 0,)

In [18]:
# creating order_hour column extrapolated from the order_date datetime info-
df['order_hour'] = df['order_date'].dt.hour

In [19]:
# creating binary order_before_noon column extrapolated from the order_hour info, marking whether it was
# placed before (1) or after (0) 12pm-
df['order_before_noon'] = np.where(df['order_hour'] < 12, 1, 0)

In [20]:
# creating order_month column extrapolated from the order_date datetime info, returns a numeric 1 thru 12-
df['order_month'] = pd.DatetimeIndex(df['order_date']).month

In [21]:
# creating order_month_name column extrapolated from the order_month info, returns an abbreviated month name
# for future use as a categorical variable-
df['order_month_name'] = df['order_month'].apply(lambda x: calendar.month_abbr[x])

In [22]:
# creating order_weekday column extrapolated from the order_date info, returns a numeric code 0 thru 6
# (Monday thru Sunday)-
df['order_weekday'] = pd.DatetimeIndex(df['order_date']).dayofweek

In [23]:
# creating order_weekday_str column from order_weekday, just retypes the 0 thru 6 code as an object for future
# use as a categorical variable- 
df['order_weekday_str'] = df['order_weekday'].astype('object')

In [24]:
# creating order_hour_str column from order_hour, just retypes the hour numeric values as an object for future
# use as a categorical variable-
df['order_hour_str'] = df['order_hour'].astype('object')

In [25]:
# checking that engineered features are displaying correctly
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180519 entries, 0 to 180518
Data columns (total 41 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   type                      180519 non-null  object        
 1   actual_ship_days          180519 non-null  int64         
 2   estimated_ship_days       180519 non-null  int64         
 3   delivery_status           180519 non-null  object        
 4   late_delivery_risk        180519 non-null  int64         
 5   category_id               180519 non-null  int64         
 6   category_name             180519 non-null  object        
 7   customer_city             180519 non-null  object        
 8   customer_id               180519 non-null  int64         
 9   customer_segment          180519 non-null  object        
 10  customer_state            180519 non-null  object        
 11  customer_zipcode          180516 non-null  float64       
 12  de

In [26]:
# create clean CSV- 
df.to_csv('../data/ship_clean.csv', index = False)